# Laboratório: Aprender a ler dados RAW com PySpark

## 1. Instale as bibliotecas necessárias (se ainda não instalou)


In [1]:
!pip install minio pandas pyarrow pyspark

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 24.7 MB/s eta 0:00:00


## 2. Importe as bibliotecas

In [2]:
from minio import Minio
import pandas as pd
import io

## 3. Configure as credenciais e endpoint do MinIO

In [3]:
minio_client = Minio(
    "minio:9000",
    access_key="4PRJYFLGzQYTnOJGH1gA",
    secret_key="ovBkCsqh2cXNkyoteCzQMV5JWCUk5tHfsG1GwYbD",
    secure=False
)

## 4. Liste os arquivos Parquet do bucket 'raw'


In [4]:
bucket_name = "raw"
objects = minio_client.list_objects(bucket_name, recursive=True)
parquet_files = [obj.object_name for obj in objects if obj.object_name.endswith('.parquet')]
print(parquet_files)

['pagila_db/actor/e11a98e7-39aa-476f-89e5-7384c2664307.parquet', 'pagila_db/address/e72da5af-ec63-4ada-bc51-6255a58541d1.parquet', 'pagila_db/category/2d384713-0c3c-4196-aad4-6631eb4f3417.parquet', 'pagila_db/city/56f840f3-df56-4e97-b42a-2e1ed958718e.parquet', 'pagila_db/country/adefc398-ff7e-4201-aca2-f00ce72e8b76.parquet', 'pagila_db/customer/ad3a6a55-81f4-4692-bce0-83bd2836f37a.parquet', 'pagila_db/customer/cb1e5090-0385-4164-8d85-67517073e6be.parquet', 'pagila_db/film/89dcd287-5502-48dc-8d23-ec375ef66258.parquet', 'pagila_db/film_actor/1c35bfc6-cd28-4b50-ae42-eb36f95e238e.parquet', 'pagila_db/film_category/6f94cad7-cb76-415c-b3e5-671e79b581f1.parquet', 'pagila_db/inventory/9b135369-f1c4-4e40-b018-03eeae50e9d6.parquet', 'pagila_db/language/450bcf2a-4841-44ea-a57f-6960eac7e05c.parquet', 'pagila_db/payment_p2022_01/e4c690fa-cc7e-475d-b269-51446175c38b.parquet', 'pagila_db/payment_p2022_02/eee8186f-001a-47ac-987a-869bca9094e6.parquet', 'pagila_db/payment_p2022_03/3a1939ed-adc8-4a20-8e5

In [7]:
# Para cada arquivo Parquet (tabela), leia os dados em um DataFrame

tables = {}
for file in parquet_files:
    response = minio_client.get_object(bucket_name, file)
    data = response.read()
    df = pd.read_parquet(io.BytesIO(data))
    #df.drop_duplicates(inplace=True)  # Remove duplicatas
    #df.dropna(inplace=True)  # Remove linhas com valores ausentes
    table_name = file.split('/')[-2]
    tables[table_name] = df
    response.close()
    response.release_conn()

# Agora, 'tabelas' é um dicionário onde a chave é o nome da tabela e o valor é o DataFrame correspondente
tables['film']

,film_id,title,description,release_year,language_id,original_language_id,rental_duration,rental_rate,length,replacement_cost,rating,last_update,special_features,fulltext
0,1,ACADEMY DINOSAUR,A Epic Drama of a Feminist And a Mad Scientist...,2006,1,NaN,6,0.99,86,20.99,PG,2022-09-10 16:46:03.905795,"b'{""Deleted Scenes"",""Behind the Scenes""}'",'academi':1 'battl':15 'canadian':20 'dinosaur...
1,2,ACE GOLDFINGER,A Astounding Epistle of a Database Administrat...,2006,1,NaN,3,4.99,48,12.99,G,2022-09-10 16:46:03.905795,"b'{Trailers,""Deleted Scenes""}'",'ace':1 'administr':9 'ancient':19 'astound':4...
2,3,ADAPTATION HOLES,A Astounding Reflection of a Lumberjack And a ...,2006,1,NaN,7,2.99,50,18.99,NC-17,2022-09-10 16:46:03.905795,"b'{Trailers,""Deleted Scenes""}'",'adapt':1 'astound':4 'baloon':19 'car':11 'fa...
3,4,AFFAIR PREJUDICE,A Fanciful Documentary of a Frisbee And a Lumb...,2006,1,NaN,5,2.99,117,26.99,G,2022-09-10 16:46:03.905795,"b'{Commentaries,""Behind the Scenes""}'",'affair':1 'chase':14 'documentari':5 'fanci':...
4,5,AFRICAN EGG,A Fast-Paced Documentary of a Pastry Chef And ...,2006,1,NaN,6,2.99,130,22.99,G,2022-09-10 16:46:03.905795,"b'{""Deleted Scenes""}'",'african':1 'chef':11 'dentist':14 'documentar...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,996,YOUNG LANGUAGE,A Unbelieveable Yarn of a Boat And a Database ...,2006,1,NaN,6,0.99,183,9.99,G,2022-09-10 16:46:03.905795,"b'{Trailers,""Behind the Scenes""}'",'administr':12 'boat':8 'boy':17 'databas':11 ...
996,997,YOUTH KICK,A Touching Drama of a Teacher And a Cat who mu...,2006,1,NaN,4,0.99,179,14.99,NC-17,2022-09-10 16:46:03.905795,"b'{Trailers,""Behind the Scenes""}'",'boat':22 'cat':11 'challeng':14 'drama':5 'ki...
997,998,ZHIVAGO CORE,A Fateful Yarn of a Composer And a Man who mus...,2006,1,NaN,6,0.99,105,10.99,NC-17,2022-09-10 16:46:03.905795,"b'{""Deleted Scenes""}'",'boy':16 'canadian':19 'compos':8 'core':2 'fa...
998,999,ZOOLANDER FICTION,A Fateful Reflection of a Waitress And a Boat ...,2006,1,NaN,5,2.99,101,28.99,R,2022-09-10 16:46:03.905795,"b'{Trailers,""Deleted Scenes""}'",'ancient':19 'boat':11 'china':20 'discov':14 ...


## 5. Criando um Warehouse no LakeKeeper

In [9]:
import requests

# Definindo a URL base da API do Lakekeeper
CATALOG_URL = "http://lakekeeper:8181/catalog"
MANAGEMENT_URL = "http://lakekeeper:8181/management"
WAREHOUSE = "trusted"

response = requests.post(f"{MANAGEMENT_URL}/v1/warehouse",
    json={
        "warehouse-name": WAREHOUSE,
        "storage-profile": {
                "type": "s3",
                "bucket": "trusted",
                "key-prefix": None,
                "assume-role-arn": None,
                "endpoint": "http://minio:9000",
                "region": "local-01",
                "path-style-access": True,
                "flavor": "s3-compat",
                "sts-enabled": False
        },
        "storage-credential": {
            "type": "s3",
            "credential-type": "access-key",
            "aws-access-key-id": "4PRJYFLGzQYTnOJGH1gA",
            "aws-secret-access-key": "ovBkCsqh2cXNkyoteCzQMV5JWCUk5tHfsG1GwYbD"
        }
    })
print(f"{response.status_code}: {response.reason}")

400: Bad Request


## 6. Criando uma tabela com PySpark e Lakekeeper

In [10]:
import pyspark
from pyspark.conf import SparkConf
from pyspark.sql import SparkSession
import pandas as pd

SPARK_VERSION = pyspark.__version__
SPARK_MINOR_VERSION = '.'.join(SPARK_VERSION.split('.')[:2])
ICEBERG_VERSION = "1.7.0"

config = {
    "spark.sql.defaultCatalog": "trusted",
    "spark.sql.catalog.trusted": "org.apache.iceberg.spark.SparkCatalog",
    "spark.sql.catalog.trusted.type": "rest",
    "spark.sql.catalog.trusted.uri": CATALOG_URL,
    "spark.sql.catalog.trusted.warehouse": WAREHOUSE,
    "spark.sql.extensions": "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions",
    "spark.jars.packages": f"""org.apache.iceberg:iceberg-spark-runtime-{SPARK_MINOR_VERSION}_2.12:{ICEBERG_VERSION},org.apache.iceberg:iceberg-aws-bundle:{ICEBERG_VERSION}"""
}


spark_config = SparkConf().setMaster('local').setAppName("Iceberg-REST")
for k, v in config.items():
    spark_config = spark_config.set(k, v)

spark = SparkSession.builder.config(conf=spark_config).getOrCreate()

spark.sql("USE trusted")

DataFrame[]

In [11]:
spark.sql(f"CREATE NAMESPACE IF NOT EXISTS pagila_db")
spark.sql("SHOW NAMESPACES").toPandas()

,namespace
0,pagila_db


## Agrupando as partições de pagamento

In [12]:
# Cria uma lista chamada payment_dfs para armazenar os DataFrames relacionados a pagamentos
payment_dfs = []

# Percorre todas as tabelas carregadas
for nome_tabela, df in tables.items():
    # Se o nome da tabela contém a palavra 'payment', adiciona o DataFrame à lista
    if 'payment' in nome_tabela:
        payment_dfs.append(df)

# Se houver algum DataFrame relacionado a 'payment', concatena todos em um único DataFrame
# Cria um DataFrame Spark a partir do DataFrame concatenado e escreve na tabela 'pagila_db.payments'
if payment_dfs:
    payments_df = pd.concat(payment_dfs, ignore_index=True)
    sdf = spark.createDataFrame(payments_df)
    sdf.writeTo("pagila_db.payments").createOrReplace()

## Criando as demais tabelas

In [13]:
# Escreve as demais tabelas normalmente
for table_name, df in tables.items():
    if 'payment' in table_name:
        continue  # já processado acima
    if not df.empty:
        sdf = spark.createDataFrame(df)
        sdf.writeTo(f"pagila_db.{table_name}").createOrReplace()
    else:
        print(f"Skipping empty DataFrame for table: {table_name}")